In [ ]:
from pathlib import Path

import numpy as np
import xarray as xr

from atm_forcing import (  # noqa: F401
    reshape_to_full_year,
)

### Save for FjordSim

In [ ]:
pattern = str(Path.home() / "NORA3" / "[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9].nc")
ds = xr.open_mfdataset(pattern, combine="by_coords")

In [ ]:
ds = reshape_to_full_year(ds)

In [ ]:
bad_steps = ds.time.diff("time") != np.timedelta64(1, "h")
print("Bad steps at positions:", bad_steps.where(bad_steps, drop=True))

In [ ]:
ds.air_temperature_2m.values = ds.air_temperature_2m.values + 273.15  # back to Kelvin

In [ ]:
ds.to_netcdf(
    Path.home() / "FjordSim_data" / "NORA3" / "NORA3.nc",
    encoding={var: {"zlib": True, "complevel": 5} for var in ds.data_vars},
)

### Saving atm parameters separately

In [ ]:
for var_name, da in ds.data_vars.items():
    da = da.rename(var_name)
    filename = f"{var_name}.nc"
    da.to_netcdf(
        Path.home() / "FjordSim_data" / "NORA3" / filename, encoding={var_name: {"zlib": True, "complevel": 5}}
    )
    print(f"{filename} saved.")